In [2]:
# Install Python libraries
!pip uninstall -y xgboost
!pip install xgboost --no-cache-dir --force-reinstall

Found existing installation: xgboost 2.1.4
Uninstalling xgboost-2.1.4:
  Successfully uninstalled xgboost-2.1.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 223.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 224.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 193.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.21.5
    Uninstalling nvidia-nccl-cu12-2.21.5:
      Successfully uninstalled nvidia-nccl-cu12-2.21.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: s

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Uninstall and reinstall NumPy cleanly
!pip uninstall -y numpy
!pip install numpy --upgrade --force-reinstall --no-cache-dir

Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Successfully uninstalled numpy-2.2.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 103.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [1]:
# Import Python libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import io
from google.colab import files
import numpy as np
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Upload CSV file
uploaded = files.upload()

Saving Redfin_Sold_OaklandCounty_OneYear.xlsx to Redfin_Sold_OaklandCounty_OneYear.xlsx


In [3]:
# Create dataframe
df = pd.read_excel('Redfin_Sold_OaklandCounty_OneYear.xlsx')

In [4]:
df.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,NaN,Single Family Residential,4835 Freer St,Rochester,MI,48306,590000.0,3.0,3.5,...,NaN,NaN,NaN,https://www.redfin.com/MI/Rochester/4835-Freer...,NaN,NaN,N,Y,42.727088,-83.154561
1,PAST SALE,January-13-2025,Condo/Co-op,1784 Flagstone Cir,Rochester,MI,48307,285000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/MI/Rochester/1784-Flags...,REALCOMP,5.805016e+10,N,Y,42.684106,-83.096150
2,PAST SALE,NaN,Condo/Co-op,3309 Silverbrook Dr,Rochester,MI,48306,454000.0,2.0,2.5,...,NaN,NaN,NaN,https://www.redfin.com/MI/Rochester/3309-Silve...,NaN,NaN,N,Y,42.728586,-83.202505
3,PAST SALE,February-21-2025,Unknown,406 S MAIN St,Rochester,MI,48307 2031,900000.0,NaN,NaN,...,Sold,NaN,NaN,https://www.redfin.com/MI/Rochester/406-S-Main...,MiRealSource-MiMLS,6.037199e+07,N,Y,42.680994,-83.133450
4,PAST SALE,February-3-2025,Single Family Residential,1219 Maple Dr,Rochester,MI,48307,390000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/MI/Rochester/1219-Maple...,REALCOMP,5.805017e+10,N,Y,42.690703,-83.125191


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17974 entries, 0 to 17973
Data columns (total 27 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   SALE TYPE                                                                                    17974 non-null  object 
 1   SOLD DATE                                                                                    15736 non-null  object 
 2   PROPERTY TYPE                                                                                17974 non-null  object 
 3   ADDRESS                                                                                      17895 non-null  object 
 4   CITY                                                                                         17903 non-null  object 
 5   STATE OR PROVINCE               

In [6]:
## DROP COLUMNS
# Columns in df
existing_columns = df.columns
# Columns to drop
columns_to_drop = [col for col in [
    'SALE TYPE',
    'STATE OR PROVINCE',
    'DAYS ON MARKET',
    'STATUS',
    'NEXT OPEN HOUSE START TIME',
    'NEXT OPEN HOUSE END TIME',
    'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
    'SOURCE',
    'MLS#',
    '$/SQUARE FEET',
    'FAVORITE',
    'INTERESTED'
] if col in existing_columns]


In [7]:
# Drop the columns
df = df.drop(columns=columns_to_drop)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17974 entries, 0 to 17973
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SOLD DATE           15736 non-null  object 
 1   PROPERTY TYPE       17974 non-null  object 
 2   ADDRESS             17895 non-null  object 
 3   CITY                17903 non-null  object 
 4   ZIP OR POSTAL CODE  17918 non-null  object 
 5   PRICE               17973 non-null  float64
 6   BEDS                16936 non-null  float64
 7   BATHS               17077 non-null  float64
 8   LOCATION            15728 non-null  object 
 9   SQUARE FEET         17176 non-null  float64
 10  LOT SIZE            14069 non-null  float64
 11  YEAR BUILT          17108 non-null  float64
 12  HOA/MONTH           7134 non-null   float64
 13  LATITUDE            17974 non-null  float64
 14  LONGITUDE           17974 non-null  float64
dtypes: float64(9), object(6)
memory usage: 2.1+ MB


In [9]:
df['PROPERTY TYPE'].unique()


array(['Single Family Residential', 'Condo/Co-op', 'Unknown',
       'Multi-Family (2-4 Unit)', 'Vacant Land',
       'Mobile/Manufactured Home', 'Multi-Family (5+ Unit)', 'Other',
       'Townhouse', 'Parking', 'Ranch'], dtype=object)

In [10]:
df['CITY'].unique()

array(['Rochester', 'Oaklandchartertownship_oakland', 'Oakland Twp',
       'Rochestercity_oakland', 'Rochester Hills', 'Washington Twp',
       'Oakland', 'Orion Twp', 'Hazel Park', 'Milford', 'Pontiac',
       'Davisburg', 'Ferndale', 'Southfield', 'Keego Harbor', 'Oxford',
       'Waterford', 'Novi', 'Oak Park', 'West Bloomfield',
       'White Lake Twp', 'Lake Orion', 'Independence Twp', 'Holly Twp',
       'West Bloomfield Twp', 'Metamora Twp', 'Bloomfield Twp', 'Wixom',
       'Madison Heights', 'Leonard', 'Addisontownship_oakland',
       'Waterford Twp', 'Rose Twp', 'White Lake', 'Royal Oak',
       'Clarkston', 'Royal Oak Twp', 'Farmington Hills', 'Auburn Hills',
       'Walled Lake', 'Whitelakechartertownship_oakland', 'Holly',
       'Highland Twp', 'Oxford Twp', 'Brandon Twp', 'South Lyon',
       'Oxford Vlg', 'Lake Orion Vlg', 'Pontiaccity_oakland',
       'Springfield Twp', 'Groveland Twp', 'Ortonville',
       'Lathrup Village', nan, 'Bloomfield Hills', 'Northville',
  

In [11]:
df['ZIP OR POSTAL CODE'].unique()

array(['48306', '48307', '48307 2031', ..., '48073 6510', '48025 1288',
       '48302 0720'], shape=(2067,), dtype=object)

In [12]:
# Remove non-residential property types
df = df[~df['PROPERTY TYPE'].isin(['Parking', 'Vacant Land', 'Unknown', 'Other', 'Multi-Family (2-4 Unit)', 'Multi-Family (5+ Unit)'])]
# Remove empty CITY
df = df[df['CITY'].notna() & (df['CITY'].str.strip() != '')]
# Remove empty PRICE
df = df[df['PRICE'].notna()]
# Trim ZIP CODE
df['ZIP OR POSTAL CODE'] = df['ZIP OR POSTAL CODE'].astype(str)
df['ZIP OR POSTAL CODE'] = df['ZIP OR POSTAL CODE'].str[:5]

In [13]:
df['ZIP OR POSTAL CODE'].unique()

array(['48306', '48307', '48363', '48030', '48341', '48340', '48075',
       '48342', '48328', '48375', '48237', '48323', '48386', '48362',
       '48393', '48033', '48442', '48076', '48302', '48383', '48073',
       '48220', '48071', '48371', '43877', '48326', '48377', '48348',
       '48359', '48336', '48350', '48309', '48304', '48324', '48380',
       '48320', '48382', '48329', '48091', '48356', '48178', '48327',
       '48083', '48165', '48346', '48334', '48085', '48067', '48374',
       '48331', '48462', '48370', '48009', '48322', '48167', '48034',
       '48390', '48367', '48335', '48357', '48360', '48072', '48017',
       '48084', '48301', '48092', '48381', '48025', '48238', '48430',
       '48439', '48539', '48098', '48069', '48317', '48070', '48836',
       '48316', '48332', '48152', '48305', '48314', '48438', '48028'],
      dtype=object)

In [14]:
# SOLD DATE IMPUTATION
# Convert Sold Date to datetime
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'], errors='coerce')

# Step 1: Fill missing SOLD DATE with mode within each ZIP code group
df['SOLD DATE'] = df.groupby('ZIP OR POSTAL CODE')['SOLD DATE'].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
)
# Step 2: Fill any remaining missing values with the overall mode
overall_mode = df['SOLD DATE'].mode()[0]
df['SOLD DATE'].fillna(overall_mode, inplace=True)

<ipython-input-14-76348e88c706>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SOLD DATE'].fillna(overall_mode, inplace=True)


In [15]:
## HOA IMPUTATION
# Fill HOA missing values based on PROPERTY TYPE and ZIP (if Property Type typically has low HOA, then 0)
df['HOA/MONTH'] = df.groupby(['ZIP OR POSTAL CODE','PROPERTY TYPE'])['HOA/MONTH'].transform(
    lambda x: x.fillna(0 if x.median() < 10 else x.median())
)
# Fill NaN with zerodf
df['HOA/MONTH'] = df.groupby('PROPERTY TYPE')['HOA/MONTH'].transform(
    lambda x: x.fillna(x.median())
)

In [16]:
## LOT SIZE IMPUTATION
# Step 1: ZIP + Property Type Median
df['LOT SIZE'] = df.groupby(['ZIP OR POSTAL CODE', 'PROPERTY TYPE'])['LOT SIZE'].transform(
    lambda x: x.fillna(x.median())
)
# Step 2: Property Type Median
df['LOT SIZE'] = df.groupby('PROPERTY TYPE')['LOT SIZE'].transform(
    lambda x: x.fillna(x.median())
)
# Step 3: Overall Median
df['LOT SIZE'].fillna(df['LOT SIZE'].median(), inplace=True)

<ipython-input-16-405b27a3b312>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LOT SIZE'].fillna(df['LOT SIZE'].median(), inplace=True)


In [17]:
## BEDS IMPUTATION
# Step 0: Replace values < 1 with NaN
df['BEDS'] = df['BEDS'].apply(lambda x: np.nan if x < 1 else x)
# Step 1: ZIP + Property Type Median
df['BEDS'] = df.groupby(['ZIP OR POSTAL CODE', 'PROPERTY TYPE'])['BEDS'].transform(
    lambda x: x.fillna(x.median())
)
# Step 2: Property Type Median
df['BEDS'] = df.groupby('PROPERTY TYPE')['BEDS'].transform(
    lambda x: x.fillna(x.median())
)
# Step 3: Overall Median
df['BEDS'].fillna(df['BEDS'].median(), inplace=True)

<ipython-input-17-0058091d74a3>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['BEDS'].fillna(df['BEDS'].median(), inplace=True)


In [18]:
## BATHROOMS IMPUTATION
# Step 0: Replace values < 1 with NaN
df['BATHS'] = df['BATHS'].apply(lambda x: np.nan if x < 1 else x)
# Step 1: ZIP + Property Type Median
df['BATHS'] = df.groupby(['ZIP OR POSTAL CODE', 'PROPERTY TYPE'])['BATHS'].transform(
    lambda x: x.fillna(x.median())
)
# Step 2: Property Type Median
df['BATHS'] = df.groupby('PROPERTY TYPE')['BATHS'].transform(
    lambda x: x.fillna(x.median())
)
# Step 3: Overall Median
df['BATHS'].fillna(df['BATHS'].median(), inplace=True)

<ipython-input-18-73e23f105091>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['BATHS'].fillna(df['BATHS'].median(), inplace=True)


In [19]:
## SQUARE FOOTAGE IMPUTATION
# Step 0: Replace values < 1 with NaN
df['SQUARE FEET'] = df['SQUARE FEET'].apply(lambda x: np.nan if x < 100 else x)
# Step 1: ZIP + Property Type Median
df['SQUARE FEET'] = df.groupby(['ZIP OR POSTAL CODE', 'PROPERTY TYPE'])['SQUARE FEET'].transform(
    lambda x: x.fillna(x.median())
)
# Step 2: Property Type Median
df['SQUARE FEET'] = df.groupby('PROPERTY TYPE')['SQUARE FEET'].transform(
    lambda x: x.fillna(x.median())
)
# Step 3: Overall Median
df['SQUARE FEET'].fillna(df['SQUARE FEET'].median(), inplace=True)

<ipython-input-19-3f96f55ccf2f>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SQUARE FEET'].fillna(df['SQUARE FEET'].median(), inplace=True)


In [20]:
## YEAR BUILT IMPUTATION
# Step 1: Step 1: ZIP + Property Type Median
df['YEAR BUILT'] = df.groupby(['ZIP OR POSTAL CODE', 'PROPERTY TYPE'])['YEAR BUILT'].transform(
    lambda x: x.fillna(x.median())
)
# Step 2: Property Type Median
df['YEAR BUILT'] = df.groupby('PROPERTY TYPE')['YEAR BUILT'].transform(
    lambda x: x.fillna(x.median())
)
# Step 3: Overall Median
df['YEAR BUILT'].fillna(df['YEAR BUILT'].median(), inplace=True)


<ipython-input-20-558f66249c9f>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YEAR BUILT'].fillna(df['YEAR BUILT'].median(), inplace=True)


In [21]:
## Feature creation
# Price Per Square Foot
PricePerSqFt = df['PRICE']/df['SQUARE FEET']
df['PricePerSqFt'] = PricePerSqFt
# HomeAge (years)
HomeAge =  df['SOLD DATE'].dt.year - df['YEAR BUILT']
df['HomeAge'] = HomeAge
# Sale Month
df['SaleMonth'] = df['SOLD DATE'].dt.month
# Sale Qtr
df['SaleQtr'] = df['SOLD DATE'].dt.quarter
# Has HOA
df['HasHOA'] = df['HOA/MONTH'].apply(lambda x: 1 if x > 0 else 0)
# Lot to Home Ratio
LotToHomeRatio = df['LOT SIZE']/df['SQUARE FEET']
df['LotToHomeRatio'] = LotToHomeRatio
# Price Per Bedroom
PricePerBedroom = df['PRICE']/df['BEDS']
df['PricePerBedroom'] = PricePerBedroom
# Price Per Bathroom
PricePerBathroom = df['PRICE']/df['BATHS']
df['PricePerBathroom'] = PricePerBathroom
# Bed to Bath Ratio
BedToBathRatio = df['BEDS']/df['BATHS']
df['BedToBathRatio'] = BedToBathRatio
# Bed x Bath
df['BedBathProduct'] = df['BEDS'] * df['BATHS']
# Luxury Home IndicatorDefine feature-based luxury criteria
sf_thresh = 3500
bath_thresh = 3.5
hoa_thresh = 250

df['IsLuxury'] = df.apply(
    lambda row: 1 if (
        row['SQUARE FEET'] >= sf_thresh and
        row['BATHS'] >= bath_thresh and
        row['HOA/MONTH'] >= hoa_thresh
    ) else 0,
    axis=1
)
# ZIP and CITY Median Price per sq ft
df['ZIP_PPSF_MEDIAN'] = df.groupby('ZIP OR POSTAL CODE')['PricePerSqFt'].transform('median')
df['CITY_PPSF_MEDIAN'] = df.groupby('CITY')['PricePerSqFt'].transform('median')


In [22]:
# KNN Neighbor to include average PricePerSqFt for five nearest homes
# 1. Coordinates and target
coords = df[['LATITUDE', 'LONGITUDE']]
target = df['PricePerSqFt']

# 2. Fit the KNN model
k = 5
knn = NearestNeighbors(n_neighbors=k + 1)  # +1 to include self
knn.fit(coords)

# 3. Find nearest neighbors for each property
distances, indices = knn.kneighbors(coords)

# 4. Compute the average PPSF of the k nearest neighbors (excluding self)
avg_ppsf_neighbors = []
for row in indices:
    neighbor_indices = row[1:]  # skip index 0 (it's the point itself)
    avg_price = target.iloc[neighbor_indices].mean()
    avg_ppsf_neighbors.append(avg_price)

# 5. Add as new feature
df['AvgPPSF_KNN'] = avg_ppsf_neighbors

In [23]:
# Remove Outliers
df = df[(df['PricePerSqFt'] > 50) & (df['PricePerSqFt'] < 500)]

In [24]:
# Frequency Encoding variables
df['CITY_ENCODED'] = df['CITY'].map(df['CITY'].value_counts())
df['ZIP_ENCODED'] = df['ZIP OR POSTAL CODE'].map(df['ZIP OR POSTAL CODE'].value_counts())
df['PROPERTY_TYPE_ENCODED'] = df['PROPERTY TYPE'].map(df['PROPERTY TYPE'].value_counts())

In [25]:
# Download the CSV file
df.to_csv('cleaned_real_estate_data.csv', index=False)
files.download('cleaned_real_estate_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Models:**
*   *Log transform PricePerSqFt dependent variable*
*   *Define Dependent variables / features*
*   *Define Hyperparameters (XGBoost Model)*
1.   Random Forest Model
2.   XGBoost Model




In [26]:
# Define Target variable
df['LOG_PPSF'] = np.log1p(df['PricePerSqFt'])
y = df['LOG_PPSF']

# Define Independent variables / features
feature_cols = [
    'BEDS', 'BATHS', 'SQUARE FEET', 'LOT SIZE', 'HOA/MONTH',
    'BedBathProduct', 'HomeAge', 'SaleQtr', 'HasHOA',
    'CITY_ENCODED', 'ZIP_ENCODED', 'PROPERTY_TYPE_ENCODED',
    'LotToHomeRatio', 'BedToBathRatio', 'IsLuxury',
    'AvgPPSF_KNN'
]
X = df[feature_cols]

# Hyper parameters
param_dist = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [3, 5, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5, 10]
}

MODEL ONE - Random Forest

In [27]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred_log = model.predict(X_test)

# Reverse log1p to get $/sqft
y_pred = np.expm1(y_pred_log)
y_actual = np.expm1(y_test)

# Calculate MAE in real-world $/sqft
mae = mean_absolute_error(y_actual, y_pred)
print(f"📉 Random Forest MAE: ${mae:,.2f}")

📉 Random Forest MAE: $32.46


MODEL TWO - XGBoost



In [28]:
# Train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Train XGBoost model
xgb_base = XGBRegressor(objective='reg:squarederror', random_state=41)

search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=25,               # Try 25 combinations
    cv=3,                    # 3-fold cross-validation
    scoring='neg_mean_absolute_error',
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

# Best model
best_xgb = search.best_estimator_
print("✅ Best params:", search.best_params_)

# Evaluate Model
y_pred_log = best_xgb.predict(X_test)

# Reverse log1p to get $/sqft
y_pred = np.expm1(y_pred_log)
y_actual = np.expm1(y_test)

# Calculate MAE in real-world $/sqft
mae = mean_absolute_error(y_actual, y_pred)
print(f"✅ XGBoost MAE: ${mae:.2f} per sq ft")

Fitting 3 folds for each of 25 candidates, totalling 75 fits
✅ Best params: {'subsample': 0.8, 'n_estimators': 400, 'min_child_weight': 5, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
✅ XGBoost MAE: $30.73 per sq ft


In [29]:
from sklearn.metrics import mean_squared_error, r2_score

rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
r2 = r2_score(y_actual, y_pred)

print(f"📉 MAE: ${mae:.2f}")
print(f"📐 RMSE: ${rmse:.2f}")
print(f"📊 R² Score: {r2:.3f}")


📉 MAE: $30.73
📐 RMSE: $44.58
📊 R² Score: 0.586


In [30]:
!pip install catboost --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [31]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Reuse existing train-test split
# X_train, X_test, y_train, y_test already defined

# Initialize and train CatBoost
cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='MAE',
    verbose=100,
    random_seed=42
)

cat_model.fit(X_train, y_train)

# Predict
y_pred_log = cat_model.predict(X_test)

# Reverse log1p to get $/sqft
y_pred = np.expm1(y_pred_log)
y_actual = np.expm1(y_test)

# Evaluate
mae = mean_absolute_error(y_actual, y_pred)
rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
r2 = r2_score(y_actual, y_pred)

print(f"🐱 CatBoost MAE: ${mae:.2f}")
print(f"🐱 RMSE: ${rmse:.2f}")
print(f"🐱 R² Score: {r2:.3f}")


0:	learn: 0.2472606	total: 52.4ms	remaining: 52.4s
100:	learn: 0.1612280	total: 1.36s	remaining: 12.1s
200:	learn: 0.1516872	total: 1.8s	remaining: 7.17s
300:	learn: 0.1448304	total: 2.25s	remaining: 5.23s
400:	learn: 0.1402344	total: 2.71s	remaining: 4.05s
500:	learn: 0.1367259	total: 3.38s	remaining: 3.37s
600:	learn: 0.1339479	total: 3.86s	remaining: 2.56s
700:	learn: 0.1317127	total: 4.76s	remaining: 2.03s
800:	learn: 0.1297237	total: 5.96s	remaining: 1.48s
900:	learn: 0.1279774	total: 8.04s	remaining: 884ms
999:	learn: 0.1265217	total: 10.2s	remaining: 0us
🐱 CatBoost MAE: $31.17
🐱 RMSE: $45.19
🐱 R² Score: 0.575


In [33]:
import pickle

# Replace best_xgb with your model variable (e.g., best_cat or model)
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_xgb, f)


In [34]:
from google.colab import files
files.download('best_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>